In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = '2'  #'0,1,2,3,4,5,6,7'
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments, ViTImageProcessor,AutoImageProcessor
import os
from PIL import Image
from sklearn.metrics import accuracy_score
import numpy as np
import pyarrow.parquet as pq
import io
from torchvision import datasets, transforms, models
from sklearn.model_selection import train_test_split
from peft import LoraConfig, TaskType, get_peft_model,PeftModel, PeftConfig
#from transformers import LoRAConfig, LoRAAdapter
from utils import get_mnist_data, get_EuroSAT_data,CustomTensorDataset, get_cifar10_data, get_car_data,get_fruits_data, get_GTSRB_data, get_DTD_data, get_resis_data, get_grabage_data, get_plants_data
import random
from tqdm import tqdm
import torch.nn.functional as F

def set_random():
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    # 如果使用 GPU，还需固定 GPU 相关随机数
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # 用于多 GPU 情况
    # 确保卷积操作等确定性（针对特定卷积算法）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def test_result():
    sum1 = 0
    set_random()
    for i in tqdm(range(min(1000, len(test_dataset)))):
        with torch.no_grad():
            outputs = model(test_dataset[i]["pixel_values"].unsqueeze(0).cuda())
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=-1).item()
        if predicted_class != int(test_dataset[i]['labels']):
            # print(predicted_class, int(test_dataset[i]['labels']))
            sum1 += 1
    print(1-sum1/min(1000, len(test_dataset)))
    return 1-sum1/min(1000, len(test_dataset))
    

def compute_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_score(labels, predictions)}
model_dir = "/mnt/data_B/lyf/vit-base-patch16-224"
model = ViTForImageClassification.from_pretrained(model_dir, torch_dtype="auto")
feature_extractor = ViTImageProcessor.from_pretrained(model_dir)

# lora_config = LoraConfig(
#         #task_type=TaskType.CAUSAL_LM,
#         #task_type=TaskType.SEQ_CLS,
#         task_type=TaskType.FEATURE_EXTRACTION,
#         target_modules=[ "intermediate.dense", "output.dense"],
#         inference_mode=False,  # 训练模式
#         r=8,  # Lora 秩
#         lora_alpha=32,  # 等效于lr=lr*lora_alpha/r
#         lora_dropout=0.1
    # )

lora_config = LoraConfig(
        r=16,
        lora_alpha=16,
        target_modules=[ "intermediate.dense", "output.dense"],  #["query", "value"]
        lora_dropout=0.1,
        bias="none",
        modules_to_save=["classifier"],
    )

model = get_peft_model(model, lora_config).cuda()
# model.print_trainable_parameters()


/home/xiaoyuanxin/miniconda3/envs/lyf_jiuan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
for name,param in model.named_parameters():
    if "lora" in name:
        param.data.fill_(0)
        print(name,param) 

In [2]:

task = "multi_task"  #0.978  # GTSRB DTD 0.737 resis 0.911  grabage 0.633  plants 0.892
def load_model_data(task):
    path_dict = {"mnist": "./mnist_checkpoint/checkpoint-20","EuroSAT": "./EuroSAT_checkpoint/checkpoint-43","cifar10": "./cifar10_checkpoint/checkpoint-16", "car":"./car_checkpoint/checkpoint-180", "fruits":"./fruits_checkpoint/checkpoint-158", "GTSRB":"./GTSRB_checkpoint/checkpoint-92", "DTD":"./DTD_checkpoint/checkpoint-61", "resis":"./resis_checkpoint/checkpoint-157", "grabage":"./grabage_checkpoint/checkpoint-25", "plants":"./plants_checkpoint/checkpoint-113"}
    task_functions = {"mnist": get_mnist_data,"EuroSAT": get_EuroSAT_data,"cifar10": get_cifar10_data,"car": get_car_data,"fruits": get_fruits_data,"GTSRB": get_GTSRB_data,"DTD": get_DTD_data,"resis": get_resis_data,"grabage": get_grabage_data, "plants": get_plants_data}
    base_model =  ViTForImageClassification.from_pretrained(model_dir, torch_dtype="auto")
    peft_config = PeftConfig.from_pretrained(path_dict[task])
    # 加载 LoRA 适配器到主模型
    model = PeftModel.from_pretrained(base_model, path_dict[task]).to("cuda:0")
    train_dataset, test_dataset = task_functions[task]()
    return model, train_dataset, test_dataset
# model, train_dataset, test_dataset = load_model_data(task)

In [ ]:
len(train_dataset), len(test_dataset), 

In [2]:
# train_dataset, test_dataset = get_GTSRB_data()  
# train_dataset, test_dataset = get_DTD_data()
# train_dataset, test_dataset = get_resis_data()  
# train_dataset, test_dataset = get_grabage_data() 
train_dataset, test_dataset = get_mnist_data()  

In [ ]:
test_result()

In [3]:
task_functions = {"mnist": get_mnist_data,"EuroSAT": get_EuroSAT_data,"cifar10": get_cifar10_data,"car": get_car_data,"fruits": get_fruits_data,"GTSRB": get_GTSRB_data,"DTD": get_DTD_data,"resis": get_resis_data,"grabage": get_grabage_data, "plants": get_plants_data}
train_dataset, test_dataset = get_mnist_data()
for j, name in enumerate(["EuroSAT", "cifar10", "car", "fruits","GTSRB", "DTD","resis", "grabage","plants"]):
    _train_dataset, _test_dataset = task_functions[name]()
    train_dataset += _train_dataset
    test_dataset += _test_dataset

In [ ]:

training_args = TrainingArguments(
    output_dir=f"./{task}_checkpoint",          # 输出目录
    #evaluation_strategy="epoch",    # 每个 epoch 后评估一次
    #save_steps=10,
    save_total_limit=1,
    learning_rate=5e-3,              # 学习率2e-5
    per_device_train_batch_size=64, # 训练批次大小
    per_device_eval_batch_size=64,  # 验证批次大小
    num_train_epochs=1,             # 训练轮数
    weight_decay=0.01,              # 权重衰减
    logging_dir="./logs",           # 日志目录
    logging_steps=10,               # 每 10 步记录一次日志
)

# for i in range(15):
trainer = Trainer(
    model=model,                         # 模型
    args=training_args,                  # 训练参数
    train_dataset=train_dataset,         # 训练集
    eval_dataset=test_dataset,            # 验证集
    tokenizer=feature_extractor,         # 特征提取器（用于处理输入）
    compute_metrics=compute_accuracy      # 评估指标
)

# 开始训练

trainer.train()
    # test_result()

In [ ]:
test_result()

In [3]:
def dropout_consist_layers(N,task):
    for layer in range(13-N):
        temp_matrix = []
        for i in range(N):
            temp_matrix.append([model.base_model.model.vit.encoder.layer[layer+i].intermediate.dense.lora_A.default.weight.clone(), model.base_model.model.vit.encoder.layer[i].output.dense.lora_A.default.weight.clone()])
            model.base_model.model.vit.encoder.layer[layer+i].intermediate.dense.lora_A.default.weight.fill_(0)
            model.base_model.model.vit.encoder.layer[layer+i].output.dense.lora_A.default.weight.fill_(0)
        result = test_result()
        with open(f'./result_Neuron-{task}.txt',"a") as f:
            f.write(f"连续去除第{layer}到{layer+N-1}层的lora参数" + "-->错了" + str(result) + " \n")
        for i in range(N):
            
            model.base_model.model.vit.encoder.layer[layer+i].intermediate.dense.lora_A.default.weight.copy_(temp_matrix[i][0])
            model.base_model.model.vit.encoder.layer[layer+i].output.dense.lora_A.default.weight.copy_(temp_matrix[i][1])
        with open(f'./result_Neuron-{task}.txt',"a") as f:
            f.write(" \n")

In [ ]:
# 神经元去除
# for task in ["mnist", "EuroSAT", "cifar10"]:
for task in [ "plants"]: #"GTSRB", "DTD","resis", "grabage"
    model, train_dataset, test_dataset = load_model_data(task)
    for i in range(1,13):
        dropout_consist_layers(i,task)

In [ ]:
model, train_dataset, test_dataset = load_model_data("fruits")
for layer in range(5,12):
    print(layer)
    model.base_model.model.vit.encoder.layer[layer].intermediate.dense.lora_A.default.weight.fill_(0)
    model.base_model.model.vit.encoder.layer[layer].output.dense.lora_A.default.weight.fill_(0)
test_result()

#dropout_layer_dict = {0:[2,2], 1:[2,2], 2:[6,11], 3:[1,1], 4:[5,11]}

In [32]:
#指定层进行模型融合-获取每个任务指定层的lora参数
def get_every_task_param():
    dropout_layer_dict = {"mnist":[2,2], "EuroSAT":[2,2], "cifar10":[6,11], "car":[1,1], "fruits":[5,11], "GTSRB":[0,0], "DTD":[0,0], "resis":[0,0], "grabage":[5,10], "plants":[11,11]}
    every_task_param = []
    for task in [ "mnist", "EuroSAT", "cifar10", "car", "fruits","GTSRB", "DTD","resis", "grabage","plants"]:
        path_dict = {"mnist": "./mnist_checkpoint/checkpoint-20","EuroSAT": "./EuroSAT_checkpoint/checkpoint-43","cifar10": "./cifar10_checkpoint/checkpoint-16", "car":"./car_checkpoint/checkpoint-180", "fruits":"./fruits_checkpoint/checkpoint-158", "GTSRB":"./GTSRB_checkpoint/checkpoint-92", "DTD":"./DTD_checkpoint/checkpoint-61", "resis":"./resis_checkpoint/checkpoint-157", "grabage":"./grabage_checkpoint/checkpoint-25", "plants":"./plants_checkpoint/checkpoint-113"}
        base_model =  ViTForImageClassification.from_pretrained(model_dir, torch_dtype="auto")
        peft_config = PeftConfig.from_pretrained(path_dict[task])
        # 加载 LoRA 适配器到主模型
        model = PeftModel.from_pretrained(base_model, path_dict[task]).to("cuda:0")
        # for layer in range(dropout_layer_dict[task][0], dropout_layer_dict[task][1]+1):
        #     model.base_model.model.vit.encoder.layer[layer].intermediate.dense.lora_A.default.weight.fill_(0)
        #     model.base_model.model.vit.encoder.layer[layer].output.dense.lora_A.default.weight.fill_(0)
        #     model.base_model.model.vit.encoder.layer[layer].intermediate.dense.lora_B.default.weight.fill_(0)
        #     model.base_model.model.vit.encoder.layer[layer].output.dense.lora_B.default.weight.fill_(0)
        lora_params = {k: v for k, v in model.state_dict().items() if 'lora'  in k and "attention" not in k}
        del model, base_model
        torch.cuda.empty_cache()
        every_task_param.append(lora_params)
    return every_task_param
every_task_param = get_every_task_param()

In [ ]:
every_task_param[0]

In [5]:
def merge(weight_list, model): 
    merge_lora_dict = dict()
    topk_vals, topk_indices = torch.topk(weight_list, k=2)
    indices = topk_indices.cpu().numpy()
    for key in every_task_param[0]:
        temp = 0 
        # weight_list = [0.2, 0.2, 0.2, 0.2, 0.2,0.2, 0.2, 0.2, 0.2, 0.2]
        # weight_list = [0.1] * 10
        for i,num in enumerate(weight_list):
        # for i,num in zip(indices,topk_vals):
            temp += num * every_task_param[i][key] 
        merge_lora_dict[key] = temp  #ours
        #merge_lora_dict[key] = temp * A
    for name,param in model.named_parameters():
        if "lora" in name and "attention" not in name:
            param.data = merge_lora_dict[name]
    return model

In [13]:
torch.tensor([0])

tensor([0])

In [37]:


# def ties_merge(model):
model, train_dataset, test_dataset = load_model_data("mnist")
every_task_param = get_every_task_param()
all_tensor_list = []
merge_lora_dict = dict()
k = 0.1
for key in every_task_param[0]:
    for i in range(len(every_task_param)):
        all_tensor_list.append(every_task_param[i][key].flatten())
        # print(every_task_param[i][key].flatten().shape)
delete_value,_ = torch.cat(all_tensor_list).abs().kthvalue(int(k*torch.cat(all_tensor_list).shape[0]))

for key in every_task_param[0]:
    sum_tensor = torch.stack([every_task_param[i][key] for i in range(len(every_task_param))], dim=0).sum(dim=0)
    # merge_lora_dict[key] = sum([every_task_param[i][key] for i in range(len(every_task_param))])/len(every_task_param)
    for i in range(len(every_task_param)):
        mask = every_task_param[i][key].abs() >= delete_value
        every_task_param[i][key] = every_task_param[i][key] * mask
        sign_match = torch.sign(sum_tensor) == torch.sign(every_task_param[i][key])
        every_task_param[i][key] = every_task_param[i][key] * sign_match
    merge_lora_dict[key] = torch.stack([every_task_param[i][key] for i in range(len(every_task_param))], dim=0).sum(dim=0)/len(every_task_param)
for name,param in model.named_parameters():
    if "lora" in name and "attention" not in name:
        param.data = merge_lora_dict[name]
    # return model


In [29]:
def ti123(model):
    return model

In [11]:
b = torch.tensor([ 9.0,  7.0, -6.0, -8.0, -1.0])/10
b

tensor([ 0.9000,  0.7000, -0.6000, -0.8000, -0.1000])

In [12]:
import torch

a = torch.tensor([ 1.0, -2.0,  3.0, -4.0,  5.0])
b = torch.tensor([ 9.0,  7.0, -6.0, -8.0, -1.0])
print(sum(a,b))
# 计算符号是否一致（都为正 or 都为负）
sign_match = torch.sign(a) == torch.sign(b)

# 保留 a 中符号一致的位置，其它位置设为 0
result = a * sign_match

print(result)


tensor([12., 10., -3., -5.,  2.])
tensor([ 1., -0.,  0., -4.,  0.])


In [17]:
import torch

tensor_list = [
    torch.tensor([[1.0, 2.0, 3.0],[1.0, 2.0, 3.0]]),
    torch.tensor([[4.0, 5.0, 6.0], [4.0, 5.0, 6.0]]),
    torch.tensor([[7.0, 8.0, 9.0],[7.0, 8.0, 9.0] ])
]

result = torch.stack(tensor_list, dim=0).sum(dim=0)/10
print(result)  # tensor([12., 15., 18.])


tensor([[1.2000, 1.5000, 1.8000],
        [1.2000, 1.5000, 1.8000]])


In [26]:
torch.cat(all_tensor_list).shape[0]

14745600

In [ ]:
model, train_dataset, test_dataset = load_model_data("fruits")
for layer in range(5, 12):
        model.base_model.model.vit.encoder.layer[layer].intermediate.dense.lora_A.default.weight.fill_(0)
        model.base_model.model.vit.encoder.layer[layer].output.dense.lora_A.default.weight.fill_(0)
        model.base_model.model.vit.encoder.layer[layer].intermediate.dense.lora_B.default.weight.fill_(0)
        model.base_model.model.vit.encoder.layer[layer].output.dense.lora_B.default.weight.fill_(0)
lora_params = {k: v for k, v in model.state_dict().items() if 'lora'  in k and "attention" not in k}
test_result()

In [10]:
def merge(weight_list, model): #分层融合
    merge_lora_dict = dict()
    dropout_layer_dict = {0:[2,2], 1:[2,2], 2:[6,11], 3:[1,1], 4:[5,11], 5:[0,0], 6:[0,0], 7:[0,0], 8:[5,10], 9:[11,11]}
    value, index = torch.max(weight_list, dim=0)
    for key in every_task_param[0]:
        temp = 0 
        layer = int(re.findall(r'\d+', key)[0])
        if  layer >= dropout_layer_dict[int(index)][0] and layer <= dropout_layer_dict[int(index)][1]:
            for i,num in enumerate(weight_list):
                temp += num * every_task_param[i][key] 
        else:
            temp = every_task_param[int(index)][key]    
        merge_lora_dict[key] = temp  #ours
        #merge_lora_dict[key] = temp * A
    for name,param in model.named_parameters():
        if "lora" in name and "attention" not in name:
            param.data = merge_lora_dict[name]
            # param.data = every_task_param[int(index)][name]
            #param.data = lora_params[name]
    return model

In [6]:
module_route = models.resnet18(pretrained=False, num_classes=10).cuda()  # 修改分类数
module_route.load_state_dict(torch.load("./router_model.pth"))

/home/xiaoyuanxin/miniconda3/envs/lyf_jiuan/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/xiaoyuanxin/miniconda3/envs/lyf_jiuan/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_70093/3672699077.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value fo

<All keys matched successfully>

In [ ]:
module_route.eval()
with torch.no_grad():
    model, train_dataset, test_dataset = load_model_data("cifar10")
    weight_list = F.softmax(module_route(test_dataset[19]["pixel_values"].unsqueeze(0).cuda())[0])


In [ ]:
every_task_param[i][key]

In [ ]:
li = [every_task_param[i]["base_model.model.vit.encoder.layer.0.output.dense.lora_A.default.weight"] for i in range(5)]
li * torch.tensor([1,1,1,1,1])

In [ ]:
import re

text = "abc123def456gh789"
numbers = int(re.findall(r'\d+', text)[0])
print(numbers)  # ['123', '456', '789']

In [ ]:
for key in every_task_param[4]:
    print(key)

In [ ]:
value, index = torch.max(weight_list, dim=0)
value, int(index)

In [ ]:
for task in [ "mnist", "EuroSAT", "cifar10", "car", "fruits","GTSRB", "DTD","resis", "grabage","plants"]:
    #for task in [ "fruits"]:
    model, train_dataset, test_dataset = load_model_data(task)

In [ ]:
merge(weight_list, model)

In [20]:
model, train_dataset, test_dataset = load_model_data("mnist")  

In [ ]:
from time import time
import re
                                                  #                                             0.936  0.98 0.978 0. 56 0.807
# for name in [ "mnist", "EuroSAT", "cifar10", "car", "fruits","GTSRB", "DTD","resis", "grabage","plants"]:   #0,53 0.69 0.91 0. 54 0.80;  
#     print(name)                                                                     选择模型单专家0.944  0.979 0.981 0.58  0.829
                                                                                  #未分层    #0.944  0.979 0.981 0.58  0.829
t1 = time() 
# merged_model, train_dataset, test_dataset = load_model_data(name)   
# merged_model = model
temp = 0                                                                           #分层   #0.944  0.982  0.983 0.59  0.829
module_route.eval()
with torch.no_grad():
    for task in [ "mnist", "EuroSAT", "cifar10", "car", "fruits","GTSRB", "DTD","resis", "grabage","plants"]:

        model, train_dataset, test_dataset = load_model_data(task)   
        every_task_param = get_every_task_param()  #ties
        all_tensor_list = []
        merge_lora_dict = dict()
        k = 0.1
        for key in every_task_param[0]:
            for i in range(len(every_task_param)):
                all_tensor_list.append(every_task_param[i][key].flatten())
                # print(every_task_param[i][key].flatten().shape)
        delete_value,_ = torch.cat(all_tensor_list).abs().kthvalue(int(k*torch.cat(all_tensor_list).shape[0]))

        for key in every_task_param[0]:
            sum_tensor = torch.stack([every_task_param[i][key] for i in range(len(every_task_param))], dim=0).sum(dim=0)
            # merge_lora_dict[key] = sum([every_task_param[i][key] for i in range(len(every_task_param))])/len(every_task_param)
            for i in range(len(every_task_param)):
                mask = every_task_param[i][key].abs() >= delete_value
                every_task_param[i][key] = every_task_param[i][key] * mask
                sign_match = torch.sign(sum_tensor) == torch.sign(every_task_param[i][key])
                every_task_param[i][key] = every_task_param[i][key] * sign_match
            # merge_lora_dict[key] = torch.stack([every_task_param[i][key] for i in range(len(every_task_param))], dim=0).sum(dim=0)/len(every_task_param)
            tensor_list =  torch.stack([every_task_param[i][key] for i in range(len(every_task_param))], dim=0)   #"max"
            merge_lora_dict[key] = tensor_list.gather(dim=0, index=tensor_list.abs().argmax(dim=0).unsqueeze(0)).squeeze(0)   #"max"
        for name,param in model.named_parameters():
            if "lora" in name and "attention" not in name:
                param.data = merge_lora_dict[name]


        # merge_model = ties_merge(model)
        # merge_model = ti123(model)
        merged_model = model
        sum1 = 0
        set_random()
        for i in tqdm(range(min(1000, len(test_dataset)))):
            # weight_list = F.softmax(module_route(test_dataset[i]["pixel_values"].unsqueeze(0).cuda())[0])
            # merged_model = merge(weight_list, model)
            with torch.no_grad():
                outputs = merged_model(test_dataset[i]["pixel_values"].unsqueeze(0).cuda())
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=-1).item()
            if predicted_class != int(test_dataset[i]['labels']):
                sum1 += 1
        print(task, 1-sum1/min(1000, len(test_dataset)))
        with open(f'./result_ft.txt',"a") as f:
            f.write(f"{round((1-sum1/min(1000, len(test_dataset))) * 100, 1)}%")
            f.write("   ")
        temp += 1-sum1/min(1000, len(test_dataset))
    with open(f'./result_ft.txt',"a") as f:
        f.write(f"{round(temp /10 * 100, 1)}%")
        f.write("\n")
    print(time()-t1)

100%|██████████| 1000/1000 [00:12<00:00, 80.54it/s]


mnist 0.724


100%|██████████| 1000/1000 [00:13<00:00, 76.29it/s]


EuroSAT 0.902


100%|██████████| 1000/1000 [00:12<00:00, 79.80it/s]


...

resis 0.819


100%|██████████| 30/30 [00:02<00:00, 14.01it/s]


grabage 0.6666666666666667


100%|██████████| 1000/1000 [00:17<00:00, 57.65it/s]

plants 0.85
145.70899844169617


In [31]:
topk_vals, topk_indices = torch.topk(weight_list, k=2)

In [ ]:
for i, num in enumerate(weight_list):
    print(i, num)
    break

In [ ]:
for i, num in zip(topk_indices.cpu().numpy(),topk_vals):
    print(i, num)
    break

In [ ]:
topk_indices.cpu().numpy()

In [ ]:
# 假设小数为 0.756
decimal_number = 0.756

# 转换为百分数并保留一位小数
percentage = round(decimal_number * 100, 1)

# 格式化输出
formatted_percentage = f"{round(decimal_number * 100, 1)}%"

print(formatted_percentage) 

In [ ]:
model, train_dataset, test_dataset = load_model_data("car")
module_route.eval()
correct = 0
total = 0

with torch.no_grad():
    for i in range():
        images, labels = test_dataset[i]["pixel_values"].unsqueeze(0).cuda(), test_dataset[i]["labels"].unsqueeze(0).cuda()
        outputs = module_route(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        for i in range(len(labels)):
            if int(preds[i]) != 3:
                print(preds[i], labels[i])
        # break
        total += labels.size(0)

acc = 100 * correct / total
print(f"Validation Accuracy: {acc:.2f}%")

In [ ]:
for name,para in model.named_parameters():
    print(name, para.shape)

In [ ]:
image = Image.open("/mnt/data_B/lyf/vision_classify_ds/GTSRB/Final_Test/Images/12626.ppm").convert('RGB')

In [ ]:
encoding = feature_extractor(images=images, return_tensors="pt")

In [11]:
import os 
import pandas as pd
df_train = pd.read_csv("/mnt/data_B/lyf/vision_classify_ds/carBrands50/train.csv")

In [ ]:

train_list = [os.path.join("/mnt/data_B/lyf/vision_classify_ds/carBrands50",i) for i in df_train["image:FILE"]]

In [ ]:
df_train

In [ ]:
(77.3+ 82+84+84.8+ 85.3)/5

In [ ]:
list(df_train["category"])

In [ ]:
for name,para in model.named_parameters():
    print(name, para.shape)

In [ ]:
import torch.version


torch.version

In [ ]:
import torch
print(torch.version.cuda)       # CUDA 版本
print(torch.backends.cudnn.version())


In [ ]:
import os
import pandas as pd 
images = []
labels = []
start_path = "/mnt/data_B/lyf/vision_classify_ds/GTSRB/Final_Training/Images/"
for i, filename in enumerate(os.listdir(start_path)):
    if filename.startswith("0"):
        for filename_ in os.listdir(os.path.join(start_path,filename)):
            if filename_.endswith("csv"):
                df = pd.read_csv(os.path.join(start_path,filename,filename_ ))
                images += [os.path.join(start_path,filename,i.split(";")[0]) for i in list(df.iloc[:, 0])]
                labels += [int(i.split(";")[-1]) for i in list(df.iloc[:, 0])]
train_list, test_list, train_labels, test_labels = train_test_split(
        images, labels, test_size=0.2, random_state=42
    )
    #     label_list.append(i)

In [ ]:
from PIL import Image
import pandas as pd
df = pd.read_csv('/mnt/data_B/lyf/vision_classify_ds/plants/plants_train.csv')
image_list = [os.path.join("/mnt/data_B/lyf/vision_classify_ds/plants", name) for name in df["Image"]]
label_list = [int(label) for label in df["CATEGORY"]]
for i,path in enumerate(image_list):
    print(i)
    Image.open(path).convert('RGB')

In [ ]:
df_test

In [ ]:
df